In [28]:
import tensorflow as tf
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [29]:
Blinks = np.load('/content/drive/MyDrive/DLA/npy/Blinks_F3.npy')
Labels = np.load('/content/drive/MyDrive/DLA/npy/Labels_F3.npy')
BlinksTest = np.load('/content/drive/MyDrive/DLA/npy/BlinksTest_F3.npy')
LabelsTest = np.load('/content/drive/MyDrive/DLA/npy/LabelsTest_F3.npy')
    #deciding the indices of each video based on the fold
    #####################Normalizing the input#############Second phase
BlinksTest[:,:,0]=(BlinksTest[:,:,0]-np.mean(Blinks[:,:,0]))/np.std(Blinks[:,:,0])
Blinks[:,:,0]=(Blinks[:,:,0]-np.mean(Blinks[:,:,0]))/np.std(Blinks[:,:,0])
    #####
    #####
BlinksTest[:,:,1]=(BlinksTest[:,:,1]-np.mean(Blinks[:,:,1]))/np.std(Blinks[:,:,1])
Blinks[:,:,1]=(Blinks[:,:,1]-np.mean(Blinks[:,:,1]))/np.std(Blinks[:,:,1])
    #####
BlinksTest[:,:,2]=(BlinksTest[:,:,2]-np.mean(Blinks[:,:,2]))/np.std(Blinks[:,:,2])
Blinks[:,:,2]=(Blinks[:,:,2]-np.mean(Blinks[:,:,2]))/np.std(Blinks[:,:,2])
    #####
BlinksTest[:,:,3]=(BlinksTest[:,:,3]-np.mean(Blinks[:,:,3]))/np.std(Blinks[:,:,3])
Blinks[:,:,3]=(Blinks[:,:,3]-np.mean(Blinks[:,:,3]))/np.std(Blinks[:,:,3])
    ####
    ####JUST TO DOUBLE CHECK
    ####
    # print(np.mean(Blinks[:,:,0]),np.mean(Blinks[:,:,1]),np.mean(Blinks[:,:,2]),np.mean(Blinks[:,:,3]))
    # print(np.std(Blinks[:,:,0]),np.std(Blinks[:,:,1]),np.std(Blinks[:,:,2]),np.std(Blinks[:,:,3]))
    # print(np.mean(BlinksTest[:,:,0]),np.mean(BlinksTest[:,:,1]),np.mean(BlinksTest[:,:,2]),np.mean(BlinksTest[:,:,3]))
    # print(np.std(BlinksTest[:,:,0]),np.std(BlinksTest[:,:,1]),np.std(BlinksTest[:,:,2]),np.std(BlinksTest[:,:,3]))

In [48]:
#BinarizationLabels
LabelsTest[LabelsTest==10] = 1
Labels[Labels==10] = 1

In [232]:
def create_model(model_name):
    if model_name == 'LSTMreza':
      input = tf.keras.Input(shape=(30, 4))
      x = tf.keras.layers.Dense(64, activation='relu')(input)
      x = tf.keras.layers.Masking(mask_value=0.0)(x)
      x = tf.keras.layers.LSTM(32, return_sequences=False,dropout=0.1, recurrent_dropout=0.1)(x)
      x = tf.keras.layers.Dense(16, activation='relu')(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(16, activation='relu')(x)
      x = tf.keras.layers.Dense(8, activation='relu')(x)
      x = tf.keras.layers.Dropout(0.1)(x)
      y = tf.keras.layers.Dense(1, activation='tanh')(x)
      model = tf.keras.models.Model(inputs=[input], outputs=[y])
    elif model_name == 'LSTM':
      input = tf.keras.Input(shape=(30, 4))
      x = tf.keras.layers.Masking(mask_value=0.0)(input)
      x = tf.keras.layers.LSTM(32, return_sequences=False,dropout=0.1, recurrent_dropout=0.1)(x)
      x = tf.keras.layers.Dense(16, activation='relu')(x)
      x = tf.keras.layers.Dropout(0.5)(x)
      y = tf.keras.layers.Dense(1, activation='tanh')(x)
      model = tf.keras.models.Model(inputs=[input], outputs=[y])
    elif model_name == 'stackedreza':
      input = tf.keras.Input(shape=(30, 4))
      x = tf.keras.layers.Dense(128, activation='relu')(input)
      x = tf.keras.layers.Masking(mask_value=0.0)(x)
      #rnn_cells = [tf.keras.layers.LSTMCell(36, dropout=0.1, recurrent_dropout=0.1) for _ in range(4)]
      #stacked_lstm = tf.keras.layers.StackedRNNCells(rnn_cells)
      #x = tf.keras.layers.RNN(stacked_lstm, return_sequences=False)(x)
      x = tf.keras.layers.LSTM(32, return_sequences=False,dropout=0.1, recurrent_dropout=0.1)(x)
      x = tf.keras.layers.Dense(32, activation='relu')(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(16, activation='relu')(x)
      x = tf.keras.layers.Dense(8, activation='relu')(x)
      x = tf.keras.layers.Dropout(0.1)(x)
      y = tf.keras.layers.Dense(1, activation='tanh')(x)
      model = tf.keras.models.Model(inputs=[input], outputs=[y])
    elif model_name == 'giaco':
      input = tf.keras.Input(shape=(30, 4))
      x = tf.keras.layers.Dense(128, activation='relu')(input)
      x = tf.keras.layers.Masking(mask_value=0.0)(x)
      #rnn_cells = [tf.keras.layers.LSTMCell(36, dropout=0.1, recurrent_dropout=0.1) for _ in range(4)]
      #stacked_lstm = tf.keras.layers.StackedRNNCells(rnn_cells)
      #x = tf.keras.layers.RNN(stacked_lstm, return_sequences=False)(x)
      x = tf.keras.layers.LSTM(64, return_sequences=False,dropout=0.1, recurrent_dropout=0.1)(x)
      x = tf.keras.layers.Dense(128, activation='relu')(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(64, activation='relu')(x)
      x = tf.keras.layers.Dense(32, activation='relu')(x)
      x = tf.keras.layers.Dense(16, activation='relu')(x)
      x = tf.keras.layers.Dense(8, activation='relu')(x)
      y = tf.keras.layers.Dense(1, activation='tanh')(x)
      model = tf.keras.models.Model(inputs=[input], outputs=[y])
    else:
      print("Chiama le cose con il loro nome")
    return model

In [284]:
model_name = "giaco"
model = create_model(model_name)

In [285]:
model.summary()

Model: "model_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_58 (InputLayer)        [(None, 30, 4)]           0         
_________________________________________________________________
dense_228 (Dense)            (None, 30, 128)           640       
_________________________________________________________________
masking_57 (Masking)         (None, 30, 128)           0         
_________________________________________________________________
lstm_36 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_229 (Dense)            (None, 128)               8320      
_________________________________________________________________
dropout_60 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_230 (Dense)            (None, 64)                825

In [286]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5)
    ]

In [287]:
lr = 0.00053
bs = 128
ep = 100
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), 
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy']
)

In [288]:
model.fit(x= Blinks, y = Labels, validation_data=(BlinksTest, LabelsTest), epochs=ep, batch_size=bs, callbacks=my_callbacks)

43/61 [====================>.........] - ETA: 2s - loss: 0.6433 - accuracy: 0.6074

KeyboardInterrupt: ignored

In [283]:
del model

## Ensembled

In [291]:
models = []
models.append(create_model('giaco'))
models.append(create_model('LSTMreza'))
models.append(create_model('LSTM'))

In [293]:
#dictionary of parameters for every net [ep, bs, lr]
model1_parameters = {'name': 'giaco', 'ep': 100, 'bs': 128, 'lr': 0.00053}
model2_parameters = {'name': 'LSTMreza','ep': 100, 'bs': 128, 'lr': 0.00053}
model3_parameters = {'name': 'LSTM','ep': 100, 'bs': 128, 'lr': 0.00053}
parameters = []
parameters.append(model1_parameters)
parameters.append(model2_parameters)
parameters.append(model3_parameters)


In [ ]:
for i in range(len(models)):
  name = parameters[i]['name']
  lr = parameters[i]['lr']
  bs = parameters[i]['bs']
  ep = parameters[i]['ep']
  models[i].compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), 
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy']
  )
  models[i].fit(x= Blinks, y = Labels, validation_data=(BlinksTest, LabelsTest), epochs=ep, batch_size=bs, callbacks=my_callbacks)
  models[i].save('/content/drive/MyDrive/DLA/' + name + 'ep=' + ep + 'bs=' + bs + 'lr=' + lr)

Epoch 1/100
61/61 [==============================] - 13s 167ms/step - loss: 1.4334 - accuracy: 0.4860 - val_loss: 0.4361 - val_accuracy: 0.7570
Epoch 2/100
23/61 [==========>...................] - ETA: 5s - loss: 0.6563 - accuracy: 0.5944

In [ ]:
#to check
labels = []
for m in models:
    predicts = np.argmax(m.predict(test), axis=1)
    labels.append(predicts)
    
# Ensemble with voting
labels = np.array(labels)
labels = np.transpose(labels, (1, 0))
labels = scipy.stats.mode(labels, axis=-1)[0]
labels = np.squeeze(labels)

In [ ]:
tf.compat.v1.disable_eager_execution()
training = tf.compat.v1.placeholder(tf.bool,name='phase_train')